In [1]:
import pandas as pd
from newest_feature_extractor import extract_features
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree, metrics
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')
from PIL import Image, ImageEnhance
import numpy as np

import tensorflow as tf

In [2]:
def prepare_mobilenet(file):
    img = tf.keras.preprocessing.image.load_img(file, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return tf.keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

def prepare_inception(file):
    img = tf.keras.preprocessing.image.load_img(file, target_size=(299, 299))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return tf.keras.applications.inception_v3.preprocess_input(img_array_expanded_dims)
def prepare_resnet(file):
    img = tf.keras.preprocessing.image.load_img(file, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return tf.keras.applications.resnet.preprocess_input(img_array_expanded_dims)

In [3]:
# Load your data into a DataFrame
data = pd.read_csv('changed_local.csv')

In [4]:
classes = {}
with open('synset_words.txt') as f:
    lines = f.readlines()
    for line in lines:
        line = line.split(' ', 1)
        classes[line[0]] = line[1].strip()
        
image_classes = {}
with open('val.txt') as f:
    lines = f.readlines()
    for line in lines:
        line = line.split(' ', 1)
        image_classes[line[0]] = classes[line[1].strip()]
        
data['class'] = list(image_classes.values())

In [5]:
# Specify the feature you want to use for stratified sampling
feature = 'average_perceived_brightness'

# Divide the data into groups based on the values of the feature
data['group'] = pd.qcut(data[feature], q=20, labels=False)

# Create a list to store the subsets
subsets = []

# Iterate over the groups and select observations from each group
for i in range(20):
    subset = data[data['group'] == i]
    subsets.append(subset)

In [6]:
for s in subsets:
    print(s['average_perceived_brightness'].min(), s['average_perceived_brightness'].max())

2.432382790232253 59.303178681175694
59.31297144074854 73.43912364510338
73.44462776187578 82.59648799103996
82.60112208934333 89.64115500492403
89.64242266376142 95.20919030404512
95.21126113097682 100.13575882405152
100.13876671883811 104.62719951229532
104.62815868121362 108.60612522876671
108.60954153233234 112.37727352607246
112.37858938641769 115.99543398632431
115.9996053607333 119.60495227860808
119.60519827735985 123.21820792828748
123.2194644024199 126.85142233059032
126.85145921376844 130.853990714776
130.8566361001562 135.41102638107444
135.4132219835315 140.78505809222884
140.7894217529512 147.73146489784472
147.73558881879825 157.26457125362336
157.27313108688372 174.16420035225318
174.17374836954127 245.67318703241892


In [7]:
# [0, 1, 2, 5, 6, 7, 10, 11, 12, 14, 15, 16, 17 18, 19]
data = data.sort_values(by='average_perceived_brightness')
# test_sets = [data[:200]]
# 19, 14, 10, 3, 1
test_sets = [subsets[0].sort_values(by='average_perceived_brightness')[:200]]

In [8]:
len(test_sets[0])

200

In [9]:
for index, s in enumerate(subsets):
    m = s[s['Mobilenet_v1'] == 1]
    i = s[s['Inception_v4'] == 1]
    r = s[s['Resnet_v1_152'] == 1]
    f = len(s) - len(m) - len(i) - len(r)
    print(f'Category {index+1}: {round(len(m)*100/len(s), 2)}% MobileNet, {round(len(i)*100/len(s), 2)}% Inception, {round(len(r)*100/len(s), 2)}% ResNet, and {round(f*100/len(s), 2)}% Failed images.')

Category 1: 67.24% MobileNet, 13.96% Inception, 4.12% ResNet, and 14.68% Failed images.
Category 2: 68.8% MobileNet, 15.08% Inception, 2.92% ResNet, and 13.2% Failed images.
Category 3: 68.48% MobileNet, 13.6% Inception, 3.64% ResNet, and 14.28% Failed images.
Category 4: 68.24% MobileNet, 14.44% Inception, 3.64% ResNet, and 13.68% Failed images.
Category 5: 68.16% MobileNet, 13.72% Inception, 3.08% ResNet, and 15.04% Failed images.
Category 6: 70.28% MobileNet, 12.92% Inception, 2.72% ResNet, and 14.08% Failed images.
Category 7: 68.52% MobileNet, 13.32% Inception, 3.56% ResNet, and 14.6% Failed images.
Category 8: 69.2% MobileNet, 13.44% Inception, 3.48% ResNet, and 13.88% Failed images.
Category 9: 67.88% MobileNet, 14.4% Inception, 3.2% ResNet, and 14.52% Failed images.
Category 10: 70.0% MobileNet, 13.96% Inception, 3.32% ResNet, and 12.72% Failed images.
Category 11: 69.6% MobileNet, 13.6% Inception, 3.44% ResNet, and 13.36% Failed images.
Category 12: 68.36% MobileNet, 14.04% In

In [10]:


mobilenet = tf.keras.applications.mobilenet.MobileNet()
resnet = tf.keras.applications.resnet.ResNet152()
inception = tf.keras.applications.inception_v3.InceptionV3()

path = 'balanced_images/original/'

In [11]:
columns = ['Image', 'Class', 'Premodel Label', 'Premodel', 'Mobilenet_v1', 'Inception_v4', 'Resnet_v1_152']
df = pd.DataFrame(columns=columns)
for index, name in enumerate(test_sets[0]['filename']):
    df.loc[index, 'Image'] = name
    


In [12]:
for index, row in test_sets[0].iterrows():
    if row['Mobilenet_v1'] == 1:
        df.loc[df['Image'] == row['filename'], 'Class'] = 1
    elif row['Inception_v4'] == 1:
        df.loc[df['Image'] == row['filename'], 'Class'] = 0
    elif row['Resnet_v1_152'] == 1:
        df.loc[df['Image'] == row['filename'], 'Class'] = 0
    else:
        df.loc[df['Image'] == row['filename'], 'Class'] = 0
df
    
        

,Image,Class,Premodel Label,Premodel,Mobilenet_v1,Inception_v4,Resnet_v1_152
0,ILSVRC2012_val_00007580.JPEG,1,NaN,NaN,NaN,NaN,NaN
1,ILSVRC2012_val_00014456.JPEG,1,NaN,NaN,NaN,NaN,NaN
2,ILSVRC2012_val_00029230.JPEG,0,NaN,NaN,NaN,NaN,NaN
3,ILSVRC2012_val_00029071.JPEG,0,NaN,NaN,NaN,NaN,NaN
4,ILSVRC2012_val_00003721.JPEG,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
195,ILSVRC2012_val_00029756.JPEG,1,NaN,NaN,NaN,NaN,NaN
196,ILSVRC2012_val_00013613.JPEG,1,NaN,NaN,NaN,NaN,NaN
197,ILSVRC2012_val_00042709.JPEG,1,NaN,NaN,NaN,NaN,NaN
198,ILSVRC2012_val_00030556.JPEG,0,NaN,NaN,NaN,NaN,NaN


## Premodel

In [13]:
from joblib import load
premodel = load('large_lr.joblib')
path = 'balanced_images/original/'

In [15]:
images = subsets[0]['filename'].tolist()
for image_index, image in enumerate(images[:100]):
    f = extract_features(path+image)
    label = premodel.predict([[f[0], f[5]]])
    if label == 1:
        print('MobileNet')
    if label == 0:
        print('Inception')

MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
Inception
MobileNet
MobileNet
Inception
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
Inception
Inception
Inception
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
Inception
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
Inception
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
Inception
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
Inception
MobileNet
MobileNet
MobileNet
Inception
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet
MobileNet


In [16]:
premodel_acc = []
p_acc = []
for s in test_sets:
    images = s['filename'].tolist()
    top1 = 0
    top5 = 0
    for image_index, image in enumerate(images):
        f = extract_features(path+image)
        label = premodel.predict([[f[0], f[5]]])
        
        if label == 1:
            preprocessed_image = prepare_mobilenet(path+image)
            predictions = mobilenet.predict(preprocessed_image)
            df.loc[df['Image'] == image, 'Premodel Label'] = 1
        if label == 0:
            preprocessed_image = prepare_inception(path+image)
            predictions = inception.predict(preprocessed_image)
            df.loc[df['Image'] == image, 'Premodel Label'] = 0
        if (image_index) % 100 == 0:
            print(f'Image #{image_index+1} --- {image}')
        results = tf.keras.applications.imagenet_utils.decode_predictions(predictions)
        labels = [el[1] for el in results[0]]
        labels = [s.replace('_', ' ') for s in labels]
        # Check Top-1
        if labels[0] in image_classes[image]:
            top1 += 1
            df.loc[df['Image'] == image, 'Premodel'] = 1
        else:
            df.loc[df['Image'] == image, 'Premodel'] = 0
        # Check Top-5

        if any(n in image_classes[image] for n in labels):
            top5 += 1
        print(labels[0], "||||", image_classes[image])
    p_acc.append((top1, top5))  

1/1 [==============================] - 6s 6s/step
Image #1 --- ILSVRC2012_val_00007580.JPEG
jack-o'-lantern |||| candle, taper, wax light
1/1 [==============================] - 0s 216ms/step
nematode |||| jellyfish
1/1 [==============================] - 2s 2s/step
spotlight |||| modem
1/1 [==============================] - 0s 235ms/step
spotlight |||| toilet seat
1/1 [==============================] - 0s 133ms/step
digital clock |||| digital clock
1/1 [==============================] - 0s 131ms/step
thimble |||| thimble
1/1 [==============================] - 0s 132ms/step
wine bottle |||| red wine
1/1 [==============================] - 0s 87ms/step
nematode |||| nematode, nematode worm, roundworm
1/1 [==============================] - 0s 98ms/step
digital clock |||| analog clock
1/1 [==============================] - 0s 132ms/step
digital clock |||| digital clock
1/1 [==============================] - 0s 208ms/step
bow tie |||| bow tie, bow-tie, bowtie
1/1 [============================

1/1 [==============================] - 0s 97ms/step
school bus |||| school bus
1/1 [==============================] - 0s 141ms/step
stage |||| accordion, piano accordion, squeeze box
1/1 [==============================] - 0s 99ms/step
platypus |||| platypus, duckbill, duckbilled platypus, duck-billed platypus, Ornithorhynchus anatinus
1/1 [==============================] - 0s 118ms/step
sax |||| electric guitar
1/1 [==============================] - 0s 133ms/step
teapot |||| hair spray
1/1 [==============================] - 0s 115ms/step
greenhouse |||| greenhouse, nursery, glasshouse
1/1 [==============================] - 0s 239ms/step
American egret |||| American egret, great white heron, Egretta albus
1/1 [==============================] - 0s 80ms/step
goblet |||| goblet
1/1 [==============================] - 0s 80ms/step
barn spider |||| barn spider, Araneus cavaticus
1/1 [==============================] - 0s 128ms/step
accordion |||| accordion, piano accordion, squeeze box
1/1 [==

1/1 [==============================] - 0s 78ms/step
feather boa |||| feather boa, boa
1/1 [==============================] - 0s 88ms/step
nematode |||| nematode, nematode worm, roundworm
1/1 [==============================] - 0s 77ms/step
perfume |||| hair spray
1/1 [==============================] - 0s 84ms/step
gasmask |||| cornet, horn, trumpet, trump
1/1 [==============================] - 0s 86ms/step
plunger |||| rifle
1/1 [==============================] - 0s 132ms/step
punching bag |||| punching bag, punch bag, punching ball, punchball
1/1 [==============================] - 0s 134ms/step
flute |||| flute, transverse flute
1/1 [==============================] - 0s 84ms/step
space shuttle |||| space shuttle
1/1 [==============================] - 0s 81ms/step
red wine |||| wine bottle
1/1 [==============================] - 0s 82ms/step
cougar |||| cougar, puma, catamount, mountain lion, painter, panther, Felis concolor
1/1 [==============================] - 0s 114ms/step
brassiere 

In [17]:
df

,Image,Class,Premodel Label,Premodel,Mobilenet_v1,Inception_v4,Resnet_v1_152
0,ILSVRC2012_val_00007580.JPEG,1,0,0,NaN,NaN,NaN
1,ILSVRC2012_val_00014456.JPEG,1,0,0,NaN,NaN,NaN
2,ILSVRC2012_val_00029230.JPEG,0,1,0,NaN,NaN,NaN
3,ILSVRC2012_val_00029071.JPEG,0,0,0,NaN,NaN,NaN
4,ILSVRC2012_val_00003721.JPEG,1,1,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...
195,ILSVRC2012_val_00029756.JPEG,1,1,1,NaN,NaN,NaN
196,ILSVRC2012_val_00013613.JPEG,1,1,0,NaN,NaN,NaN
197,ILSVRC2012_val_00042709.JPEG,1,1,1,NaN,NaN,NaN
198,ILSVRC2012_val_00030556.JPEG,0,1,1,NaN,NaN,NaN


## MobileNet

In [18]:
mnet_acc = []

for s in test_sets:
    images = s['filename'].tolist()
    top1 = 0
    top5 = 0
    for image_index, image in enumerate(images):
        if (image_index) % 100 == 0:
            print(f'Image #{image_index+1} --- {image}')
        preprocessed_image = prepare_mobilenet(path+image)
        predictions = mobilenet.predict(preprocessed_image)
        results = tf.keras.applications.imagenet_utils.decode_predictions(predictions)
        labels = [el[1] for el in results[0]]
        labels = [s.replace('_', ' ') for s in labels]
        # Check Top-1
        if labels[0] in image_classes[image]:
            top1 += 1
            df.loc[df['Image'] == image, 'Mobilenet_v1'] = 1
        else:
            df.loc[df['Image'] == image, 'Mobilenet_v1'] = 0
        # Check Top-5

        if any(n in image_classes[image] for n in labels):
            top5 += 1
            
        print(labels[0], "||||", image_classes[image])
    mnet_acc.append((top1, top5))

Image #1 --- ILSVRC2012_val_00007580.JPEG
1/1 [==============================] - 0s 164ms/step
jack-o'-lantern |||| candle, taper, wax light
1/1 [==============================] - 0s 96ms/step
nematode |||| jellyfish
1/1 [==============================] - 0s 147ms/step
spotlight |||| modem
1/1 [==============================] - 0s 134ms/step
washbasin |||| toilet seat
1/1 [==============================] - 0s 92ms/step
digital clock |||| digital clock
1/1 [==============================] - 0s 128ms/step
thimble |||| thimble
1/1 [==============================] - 0s 96ms/step
wine bottle |||| red wine
1/1 [==============================] - 0s 98ms/step
nematode |||| nematode, nematode worm, roundworm
1/1 [==============================] - 0s 93ms/step
digital clock |||| analog clock
1/1 [==============================] - 0s 165ms/step
digital clock |||| digital clock
1/1 [==============================] - 0s 124ms/step
candle |||| bow tie, bow-tie, bowtie
1/1 [==========================

1/1 [==============================] - 0s 82ms/step
school bus |||| school bus
1/1 [==============================] - 0s 85ms/step
stage |||| accordion, piano accordion, squeeze box
1/1 [==============================] - 0s 88ms/step
platypus |||| platypus, duckbill, duckbilled platypus, duck-billed platypus, Ornithorhynchus anatinus
1/1 [==============================] - 0s 139ms/step
sax |||| electric guitar
1/1 [==============================] - 0s 115ms/step
teapot |||| hair spray
1/1 [==============================] - 0s 128ms/step
greenhouse |||| greenhouse, nursery, glasshouse
1/1 [==============================] - 0s 137ms/step
valley |||| American egret, great white heron, Egretta albus
1/1 [==============================] - 0s 90ms/step
goblet |||| goblet
1/1 [==============================] - 0s 171ms/step
barn spider |||| barn spider, Araneus cavaticus
1/1 [==============================] - 0s 89ms/step
accordion |||| accordion, piano accordion, squeeze box
1/1 [===========

1/1 [==============================] - 0s 216ms/step
feather boa |||| feather boa, boa
1/1 [==============================] - 0s 242ms/step
nematode |||| nematode, nematode worm, roundworm
1/1 [==============================] - 0s 204ms/step
perfume |||| hair spray
1/1 [==============================] - 0s 218ms/step
gasmask |||| cornet, horn, trumpet, trump
1/1 [==============================] - 0s 219ms/step
plunger |||| rifle
1/1 [==============================] - 0s 215ms/step
punching bag |||| punching bag, punch bag, punching ball, punchball
1/1 [==============================] - 0s 219ms/step
flute |||| flute, transverse flute
1/1 [==============================] - 0s 219ms/step
space shuttle |||| space shuttle
1/1 [==============================] - 0s 238ms/step
red wine |||| wine bottle
1/1 [==============================] - 0s 133ms/step
cougar |||| cougar, puma, catamount, mountain lion, painter, panther, Felis concolor
1/1 [==============================] - 0s 149ms/step
br

In [19]:
mnet_acc

[(135, 171)]

## Inception

In [20]:
for s in test_sets:
    images = s['filename'].tolist()
    top1 = 0
    top5 = 0
    for image_index, image in enumerate(images):
        if (image_index) % 100 == 0:
            print(f'Image #{image_index+1} --- {image}')
        preprocessed_image = prepare_inception(path+image)
        predictions = inception.predict(preprocessed_image)
        results = tf.keras.applications.imagenet_utils.decode_predictions(predictions)
        labels = [el[1] for el in results[0]]
        labels = [s.replace('_', ' ') for s in labels]
        # Check Top-1
        if labels[0] in image_classes[image]:
            top1 += 1
            df.loc[df['Image'] == image, 'Inception_v4'] = 1
        else:
            df.loc[df['Image'] == image, 'Inception_v4'] = 0
        # Check Top-5

        if any(n in image_classes[image] for n in labels):
            top5 += 1
    inc_acc.append((top1, top5))

Image #1 --- ILSVRC2012_val_00007580.JPEG
1/1 [==============================] - 0s 449ms/step
Image #101 --- ILSVRC2012_val_00042639.JPEG
1/1 [==============================] - 0s 225ms/step


1/1 [==============================] - 0s 493ms/step


NameError: name 'inc_acc' is not defined

In [ ]:
inc_acc

## ResNet

In [23]:
res_acc = []

for s in test_sets:
    images = s['filename'].tolist()
    top1 = 0
    top5 = 0
    for image_index, image in enumerate(images):
        if (image_index) % 100 == 0:
            print(f'Image #{image_index+1} --- {image}')
        preprocessed_image = prepare_resnet(path+image)
        predictions = resnet.predict(preprocessed_image)
        results = tf.keras.applications.imagenet_utils.decode_predictions(predictions)
        labels = [el[1] for el in results[0]]
        labels = [s.replace('_', ' ') for s in labels]
        # Check Top-1
        if labels[0] in image_classes[image]:
            top1 += 1
            df.loc[df['Image'] == image, 'Resnet_v1_152'] = 1
        else:
            df.loc[df['Image'] == image, 'Resnet_v1_152'] = 0
        # Check Top-5

        if any(n in image_classes[image] for n in labels):
            top5 += 1
    res_acc.append((top1, top5))

Image #1 --- ILSVRC2012_val_00007580.JPEG
1/1 [==============================] - 1s 932ms/step
Image #101 --- ILSVRC2012_val_00042639.JPEG
1/1 [==============================] - 1s 508ms/step


1/1 [==============================] - 1s 824ms/step


In [60]:
p_a = df['Class'] == df['Premodel Label']

In [43]:
df['Premodel-Inception'] = (df['Premodel'] == df['Inception_v4'])

df[df['Premodel-Inception'] == False][['Image', 'Premodel', 'Inception_v4', 'Premodel-Inception']]
# df[['Premodel', 'Inception_v4', 'Premodel-Inception']]

,Image,Premodel,Inception_v4,Premodel-Inception
19,ILSVRC2012_val_00049138.JPEG,1,0,False
33,ILSVRC2012_val_00040843.JPEG,1,0,False
45,ILSVRC2012_val_00006882.JPEG,1,0,False
60,ILSVRC2012_val_00035427.JPEG,0,1,False
71,ILSVRC2012_val_00041246.JPEG,0,1,False
93,ILSVRC2012_val_00015961.JPEG,0,1,False
100,ILSVRC2012_val_00042639.JPEG,1,0,False
101,ILSVRC2012_val_00034468.JPEG,0,1,False
106,ILSVRC2012_val_00036856.JPEG,0,1,False
110,ILSVRC2012_val_00001547.JPEG,0,1,False


In [39]:
df['Premodel-Accuracy'] = (df['Premodel Label'] == df['Class'])
df

,Image,Class,Premodel Label,Premodel,Mobilenet_v1,Inception_v4,Resnet_v1_152,Premodel-Inception,Premodel-Accuracy
0,ILSVRC2012_val_00007580.JPEG,1,0,0,0,0,1,True,False
1,ILSVRC2012_val_00014456.JPEG,1,0,0,0,0,0,True,False
2,ILSVRC2012_val_00029230.JPEG,0,1,0,0,0,0,True,False
3,ILSVRC2012_val_00029071.JPEG,0,0,0,0,0,0,True,True
4,ILSVRC2012_val_00003721.JPEG,1,1,1,1,1,1,True,True
...,...,...,...,...,...,...,...,...,...
195,ILSVRC2012_val_00029756.JPEG,1,1,1,1,0,0,False,True
196,ILSVRC2012_val_00013613.JPEG,1,1,0,0,0,0,True,True
197,ILSVRC2012_val_00042709.JPEG,1,1,1,1,1,1,True,True
198,ILSVRC2012_val_00030556.JPEG,0,1,1,1,0,0,False,False


In [46]:
df[['Image', 'Premodel', 'Inception_v4']]

,Image,Premodel,Inception_v4
0,ILSVRC2012_val_00007580.JPEG,0,0
1,ILSVRC2012_val_00014456.JPEG,0,0
2,ILSVRC2012_val_00029230.JPEG,0,0
3,ILSVRC2012_val_00029071.JPEG,0,0
4,ILSVRC2012_val_00003721.JPEG,1,1
...,...,...,...
195,ILSVRC2012_val_00029756.JPEG,1,0
196,ILSVRC2012_val_00013613.JPEG,0,0
197,ILSVRC2012_val_00042709.JPEG,1,1
198,ILSVRC2012_val_00030556.JPEG,1,0
